In [2]:
import os
import time
import random
import copy
import sys

import numpy as np
import torch

sys.path.append("/jet/home/azhang19/stat 260/STAT-260-Final-Project-Randomized-Linear-Algebra-Transformer")
from RLALLaMA3.LLaMA3 import ModelArgs, Transformer
from RLALLaMA3.utils import (
    linear_warmup_cosine_decay_multiplicative,
    name_args,
    Args,
)
from RLALLaMA3.tasks import single_answer_seq_loss, get_dataset

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

torch.set_float32_matmul_precision("high")
device = "cuda" if torch.cuda.is_available() else "cpu"

time_stamp = time.strftime("%Y-%m-%d %H-%M-%S", time.localtime())
print(f"Time stamp: {time_stamp}")

Time stamp: 2025-05-09 02-44-26


In [29]:
import torch

# Example tensors
A = torch.arange(60).reshape(3, 4, 5)
index = torch.tensor([[0, 2],   # For batch 0, select rows 0 and 2
                      [1, 3],   # For batch 1, select rows 1 and 3
                      [0, 1]])  # For batch 2, select rows 0 and 1
                      # Shape: (3, 2)

# Ensure index tensor is of dtype long for indexing
index = index.long()

# 1. Create batch indices
# Gets the batch size from A
batch_size = A.size(0)
# Creates a tensor [0, 1, 2, ..., batch_size-1]
batch_indices = torch.arange(batch_size, device=A.device)
# Unsqueeze to make it a column vector (B, 1) for broadcasting, e.g., [[0], [1], [2]]
batch_indices = batch_indices.unsqueeze(1) # Shape: (3, 1)

# 2. Perform advanced indexing
# A[batch_indices, index] will select:
# For each b in 0..B-1, and for each k in 0..K-1:
# A[batch_indices[b,0], index[b,k], :]
# Since batch_indices broadcasts from (B,1) to (B,K) and index is (B,K),
# this effectively means A[b, index[b,k], :]
result = A[batch_indices, index]

# The shape of 'result' will be (3, 2, 5)
print("Tensor A:\n", A)
print("Index tensor:\n", index)
print("Result using advanced indexing:\n", result)
print("Shape of the result:", result.shape)

Tensor A:
 tensor([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19]],

        [[20, 21, 22, 23, 24],
         [25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34],
         [35, 36, 37, 38, 39]],

        [[40, 41, 42, 43, 44],
         [45, 46, 47, 48, 49],
         [50, 51, 52, 53, 54],
         [55, 56, 57, 58, 59]]])
Index tensor:
 tensor([[0, 2],
        [1, 3],
        [0, 1]])
Result using advanced indexing:
 tensor([[[ 0,  1,  2,  3,  4],
         [10, 11, 12, 13, 14]],

        [[25, 26, 27, 28, 29],
         [35, 36, 37, 38, 39]],

        [[40, 41, 42, 43, 44],
         [45, 46, 47, 48, 49]]])
Shape of the result: torch.Size([3, 2, 5])


In [55]:
A = torch.randn(2, 4, 5)
B = torch.randn(2, 5, 5)

In [56]:
random_tensor = torch.rand(2, 5)
permutations = torch.argsort(random_tensor, dim=1, descending=False)

In [57]:
part1 = permutations[:, :2]
part2 = permutations[:, 2:]


In [66]:
batch_indices = torch.arange(A.size(0), device=A.device).unsqueeze(1)
A_part1 = A.transpose(-1, -2)[batch_indices, part1].transpose(-1, -2)
A_part2 = A.transpose(-1, -2)[batch_indices, part2].transpose(-1, -2)

B_part1 = B[batch_indices, part1]
B_part2 = B[batch_indices, part2]



In [67]:
A_part1.shape

torch.Size([2, 4, 2])

In [64]:
B_part2.shape

torch.Size([2, 3, 5])

In [61]:
A

tensor([[[ 0.4267, -0.0805, -0.1317, -1.2567,  0.0193],
         [ 1.6972, -1.0939,  0.1474, -2.0800, -0.0226],
         [ 0.0805, -0.2762, -0.2431,  0.0344,  0.5473],
         [-1.0700, -1.3505,  0.3415, -0.5943,  0.2439]],

        [[-0.2898, -0.4877,  0.9536,  0.5764,  0.1351],
         [ 0.6834, -0.2341,  1.9765,  0.0505,  0.7867],
         [ 1.2800,  1.4581,  0.4962, -1.0274, -0.5660],
         [ 1.2235,  2.9338,  0.5091,  0.5528, -2.0385]]])